### Import libraries

In [ ]:
import os
import re
import boto3
import pandas as pd
import json
import sagemaker
import pyspark
import gzip
import s3fs
from io import StringIO, BytesIO
from pyspark.sql.functions import *

### Read files and convert to spark data frame

In [ ]:
data_2005 = pd.read_csv('https://s3.amazonaws.com/mrinal-ml-sagemaker/Project/2005_data.csv', dtype=object, nrows=100)
col_names = data_2005.columns.values
data_2005[col_names] = data_2005[col_names].astype(str) # This makes it compatible with spark dataframes

In [ ]:
sc = pyspark.SparkContext()
sql = pyspark.SQLContext(sc)

In [ ]:
spark_df_previous = sql.createDataFrame(data_2005)

In [ ]:
for i in range(6, 10):
    data = pd.read_csv('https://s3.amazonaws.com/mrinal-ml-sagemaker/Project/200' + str(i) + '_data.csv', dtype=object, nrows=100)
    col_names = data.columns.values
    data[col_names] = data[col_names].astype(str)
    spark_data = sql.createDataFrame(data)
    spark_df_union = spark_df_previous.union(spark_data)
    spark_df_previous = spark_df_union

for i in range(10, 16):
    data = pd.read_csv('https://s3.amazonaws.com/mrinal-ml-sagemaker/Project/20' + str(i) + '_data.csv', dtype=object, nrows=100)
    col_names = data.columns.values
    data[col_names] = data[col_names].astype(str)
    spark_data = sql.createDataFrame(data)
    spark_df_union = spark_df_previous.union(spark_data)

### Clean spark data frames

In [ ]:
spark_df_union = spark_df_union.drop('record_condition_20' , 'entity_condition_20', 'entity_condition_19', 
                                     'entity_condition_18', 'entity_condition_17', 'record_condition_19', 
                                     'record_condition_18', 'record_condition_16', 'record_condition_17',
                                     'record_condition_15','record_condition_14','record_condition_13',
                                     'record_condition_12','record_condition_11','record_condition_10',
                                     'record_condition_9','record_condition_8','record_condition_7' , 'record_condition_6', 
                                     'record_condition_5', 'entity_condition_16', 'entity_condition_15','entity_condition_14', 
                                     'entity_condition_13', 'entity_condition_12', 'entity_condition_11', 'entity_condition_10', 
                                     'entity_condition_9','entity_condition_8', 'entity_condition_7', 'entity_condition_6', 
                                     'entity_condition_5')

In [ ]:
spark_df_union = spark_df_union.fillna({'place_of_injury_for_causes_w00_y34_except_y06_and_y07_' : "10"})
spark_df_union = spark_df_union.fillna({'130_infant_cause_recode' : "000"})
spark_df_union = spark_df_union.fillna({'activity_code' : "10"})

In [ ]:
spark_df_union = spark_df_union.withColumn('method_of_disposition', regexp_replace('method_of_disposition', 'R' , 'O'))
spark_df_union = spark_df_union.withColumn('method_of_disposition', regexp_replace('method_of_disposition', 'E' , 'O'))
spark_df_union = spark_df_union.withColumn('method_of_disposition', regexp_replace('method_of_disposition', 'D' , 'O'))

In [ ]:
spark_df_union = spark_df_union.fillna({'manner_of_death' : "8"})

In [ ]:
# Needs explanation
spark_df_union = spark_df_union.withColumn('Place_of_death_and_decedents_status', 
                         when(spark_df_union['Place_of_death_and_decedents_status']== 9 , 7).otherwise(spark_df_union['Place_of_death_and_decedents_status']))

In [ ]:
cleaned_df = spark_df_union.toPandas()

In [ ]:
s3_path = 's3://budt758b-manav94/project/cleaned_df.csv'
bytes_to_write = spark_df_union.toPandas().to_csv(None).encode()
fs = s3fs.S3FileSystem(key=key, secret=secret_key)
with fs.open(s3_path, 'wb') as f:
    f.write(bytes_to_write)